In [1]:
import os
import string
import sys

import numpy as np
import pandas as pd

In [7]:
weather_data = pd.read_csv("data/HistoricalWeatherData/Cariboo_Quesnel/en_climate_daily_BC_1096631_2010_P1D.csv")
weather_data = weather_data[["
weather_data.head()

,Date/Time,Year,Month,Day,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,...,Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Precip (mm),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,2010-01-01,2010,1,1,-8.9,NaN,-16.3,NaN,-12.6,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,1.1,NaN,<31,NaN
1,2010-01-02,2010,1,2,-6.2,NaN,-13.4,NaN,-9.8,NaN,...,NaN,0.0,NaN,NaN,M,NaN,0.4,NaN,<31,NaN
2,2010-01-03,2010,1,3,-6.3,NaN,-18.9,NaN,-12.6,NaN,...,NaN,0.0,NaN,NaN,M,NaN,0.2,NaN,<31,NaN
3,2010-01-04,2010,1,4,-11.7,NaN,-19.6,NaN,-15.7,NaN,...,NaN,0.0,NaN,NaN,M,NaN,0.0,NaN,<31,NaN
4,2010-01-05,2010,1,5,-15.5,NaN,-26.4,NaN,-21.0,NaN,...,NaN,0.0,NaN,NaN,M,NaN,0.0,NaN,<31,NaN
